In [1]:
import io
import ftplib
import pandas as pd
import config
import json
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def get_chave_notas():
    today = date.today().strftime('%d-%m-%Y')
    yesterday = (date.today() - timedelta(1)).strftime('%Y-%m-%d')
    file_bytes = io.BytesIO()
    ocorrencias = ['114', '113']

    inectar_ftp = ftplib.FTP(config.HOSTNAME_INECTAR, config.USERNAME_INECTAR, config.PASSWORD_INECTAR)
    inectar_ftp.cwd(config.PATH_TRIMESTRAL)
    list_files = inectar_ftp.nlst()

    for file in list_files:
        if not today in file: continue
        inectar_ftp.retrbinary('RETR ' + file, file_bytes.write)
        df = pd.read_excel(file_bytes)
    inectar_ftp.close()

    df.loc[:,"Pagador do frete/Nome Fantasia"] = df["Pagador do frete/Nome Fantasia"].str.contains("MAGAZINE")
    df = df.drop_duplicates(subset=["Nota Fiscal"])
    df = df[df["Código Ocorrência"].astype(str).isin(ocorrencias)]
    df.loc[:,"Data da ocorrência"] = df["Data da ocorrência"].astype(str).str.split().str[0]
    df = df[df["Data da ocorrência"] == yesterday]
    chave_nota = df["Chave NF-e"].to_list()
    return chave_nota

In [3]:
count = 0

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 600)

driver.maximize_window()
driver.get('https://lite.arquivei.com.br/batch')

login = driver.find_element(By.XPATH, '//*[@id="email"]')
login.send_keys('daniel@conectacargo.com.br')

password = driver.find_element(By.XPATH, '//*[@id="password"]')
password.send_keys('conecta123')

iframe = driver.find_element(By.XPATH, '//*[@id="google-recaptcha"]/div/div/iframe')
driver.switch_to.frame(iframe)

recaptcha = driver.find_element(By.XPATH, '//*[@id="recaptcha-anchor"]/div[1]')
recaptcha.click()

login_botton = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="arquivei-sign-in"]/div[2]/form/div[4]/button[2]')))
login_botton.click()

list_notas = get_chave_notas()
list_downloaded = []

for nota in list_notas:
    if nota in list_downloaded: continue
    search_nota = driver.find_element(By.XPATH, '//*[@id="batch-access-key-input"]/textarea')
    search_nota.send_keys(nota)
    search_nota.send_keys('/n')
    count += 1

    if count == 300:
        download_button = driver.find_element(By.XPATH, '//*[@id="batch-send"]/button')



TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00D64DE3+43907]
	(No symbol) [0x00CF0741]
	(No symbol) [0x00BE33ED]
	(No symbol) [0x00C16760]
	(No symbol) [0x00C16C2B]
	(No symbol) [0x00C46F62]
	(No symbol) [0x00C32BA4]
	(No symbol) [0x00C455CA]
	(No symbol) [0x00C32956]
	(No symbol) [0x00C0E17E]
	(No symbol) [0x00C0F32D]
	GetHandleVerifier [0x01015AF9+2865305]
	GetHandleVerifier [0x0105E78B+3163435]
	GetHandleVerifier [0x01058441+3138017]
	GetHandleVerifier [0x00DEE0F0+605840]
	(No symbol) [0x00CFA64C]
	(No symbol) [0x00CF6638]
	(No symbol) [0x00CF675F]
	(No symbol) [0x00CE8DB7]
	BaseThreadInitThunk [0x76957BA9+25]
	RtlInitializeExceptionChain [0x7774BD3B+107]
	RtlClearBits [0x7774BCBF+191]
